In [ ]:
import os
import pandas as pd
import pickle

In [ ]:
df_train=pd.read_csv('/content/drive/MyDrive/Equation Solver/train_final.csv',index_col=False)
labels=df_train[['784']]

In [ ]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,0,0,35,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,244,219,219,219,219,219,219,219,219,219,219,219,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,177,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import keras
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense,Input

In [ ]:
model = Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='avg', input_tensor=Input(shape=(28,28,3))))
model.add(Dense(13,activation='softmax'))
model.layers[0].trainable = False


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 13)                26637     
Total params: 23,614,349
Trainable params: 26,637
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
labels=np.array(labels)
train_labels = to_categorical(labels,num_classes=13)

In [ ]:
import cv2
train_X= np.array(df_train.iloc[:,0:])

In [ ]:
train_X.shape

(47504, 784)

In [ ]:
train_X=np.dstack([train_X] * 3)
train_X.shape

(47504, 784, 3)

In [ ]:
train_X = train_X.reshape(-1, 28,28,3)
train_X.shape

(47504, 28, 28, 3)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
from keras.models import model_from_json

In [ ]:
history = model.fit(train_X, train_labels, epochs=10, batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
238/238 [==============================] - 7s 21ms/step - loss: 1.2701 - accuracy: 0.6673
Epoch 2/10
238/238 [==============================] - 5s 21ms/step - loss: 0.2397 - accuracy: 0.9350
Epoch 3/10
238/238 [==============================] - 5s 21ms/step - loss: 0.1606 - accuracy: 0.9585
Epoch 4/10
238/238 [==============================] - 5s 22ms/step - loss: 0.1242 - accuracy: 0.9672
Epoch 5/10
238/238 [==============================] - 5s 21ms/step - loss: 0.1006 - accuracy: 0.9745
Epoch 6/10
238/238 [==============================] - 5s 22ms/step - loss: 0.0849 - accuracy: 0.9797
Epoch 7/10
238/238 [==============================] - 5s 22ms/step - loss: 0.0724 - accuracy: 0.9832
Epoch 8/10
238/238 [==============================] - 5s 22ms/step - loss: 0.0649 - accuracy: 0.9856
Epoch 9/10
238/238 [==============================] - 5s 22ms/step - loss: 0.0564 - accuracy: 0.9879
Epoch 10/10
238/238 [==============================] - 5s 22ms/step - loss: 0.0502 - accura

In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Equation Solver/model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/Equation Solver/model_final.h5")